# Approximating Shapley Values

In [1]:
import matplotlib.pyplot as plt
import warnings

plt.style.use("style.mplstyle")
warnings.filterwarnings('ignore')

## Load and Prepare Data

In [2]:
import pandas as pd

features = ["% working class", "number of rooms", "NOX concentration"]
df = pd.read_csv("data.csv")
y = df["y"].values
print(f"{len(y)} rows")
print(df[features + ["y"]].sample(5, random_state=0))

# returns:
# 506 rows
#      % working class  number of rooms  NOX concentration        y
# 329            14.68            6.333              0.460  22600.0
# 371            19.06            6.216              0.631  50000.0
# 219            21.00            6.373              0.550  23000.0
# 403            39.54            5.349              0.693   8300.0
# 78             24.68            6.232              0.437  21200.0

506 rows
     % working class  number of rooms  NOX concentration        y
329            14.68            6.333              0.460  22600.0
371            19.06            6.216              0.631  50000.0
219            21.00            6.373              0.550  23000.0
403            39.54            5.349              0.693   8300.0
78             24.68            6.232              0.437  21200.0


## Train Model

In [3]:
from sklearn.ensemble import RandomForestRegressor

X = df[features]
model = RandomForestRegressor(random_state=0).fit(X, y)
y_pred = m.predict(X)

## Approximate Shapley Values

In [ ]:
def marginalised_prediction(row, m=model, X=X, missing=[]):
    """Generate a prediction for `row` using model `m`, replacing
    features in `missing` by sampling randomly from `X`."""
    instance = row.copy()
    for feature in missing:
        instance[feature] = np.random.choice(X[feature])
    return m.predict([instance])

In [4]:
import numpy as np

def approximate_prediction(row, m=model, X=X, missing=[], n=100):
    """Average the results returned by `marginalised_prediction()`
    over `n` predictions."""
    predictions = []
    for _ in range(n):
        predictions.append(marginalised_prediction(m, X, row, missing))
    return np.mean(predictions)

In [5]:
predictions = {}

# predictions with no features
predictions["none"] = X.apply(
    lambda row: approximate_prediction(
        row,
        missing=features
    ),
    axis=1
)

# predictions with one feature
for feat in features:
    predictions[feat] = X.apply(
        lambda row: approximate_prediction(
            row,
            missing=[c for c in features if c != feat]
        ),
        axis=1
    )
    
# predictions with two features
for i, feat1 in enumerate(features):
    for feat2 in features[i+1:]:
        predictions[f"{feat1}, {feat2}"] = X.apply(
            lambda row: approximate_prediction(
                row,
                missing=[c for c in features if c not in [feat1, feat2]]
            ),
            axis=1
        )
        
# predictions with all features
predictions["all"] = m.predict(X)

In [6]:
sv_pwc = 1/3 * (predictions["% working class"] - 
                predictions["none"]) +\
         1/6 * (predictions["% working class, number of rooms"] - 
                predictions["number of rooms"]) +\
         1/6 * (predictions["% working class, NOX concentration"] - 
                predictions["NOX concentration"]) +\
         1/3 * (predictions["all"] - 
                predictions["number of rooms, NOX concentration"])

sv_nor = 1/3 * (predictions["number of rooms"] - 
                predictions["none"]) +\
         1/6 * (predictions["% working class, number of rooms"] - 
                predictions["% working class"]) +\
         1/6 * (predictions["number of rooms, NOX concentration"] - 
                predictions["NOX concentration"]) +\
         1/3 * (predictions["all"] - 
                predictions["% working class, NOX concentration"])

sv_nc  = 1/3 * (predictions["NOX concentration"] - 
                predictions["none"]) +\
         1/6 * (predictions["% working class, NOX concentration"] - 
                predictions["% working class"]) +\
         1/6 * (predictions["number of rooms, NOX concentration"] - 
                predictions["number of rooms"]) +\
         1/3 * (predictions["all"] - 
                predictions["% working class, number of rooms"])

In [7]:
print("Mean absolute shapley values:")
print(f"% working class  : {np.abs(sv_pwc).mean():,.1f}")
print(f"number of rooms  : {np.abs(sv_nor).mean():,.1f}")
print(f"NOX concentration: {np.abs(sv_nc).mean():,.1f}")

Mean absolute shapley values:
% working class  : 4,051.3
number of rooms  : 2,695.0
NOX concentration: 1,289.9
